<a href="https://colab.research.google.com/github/GraceChen996/TensorFlowLearningNote/blob/main/example2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
!pip install gym[box2d]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [75]:
import gym
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras
from tensorflow.keras.layers import Dense 

In [60]:


class PolicyGradientNetwork(keras.Model):
    def __init__(self, n_actions, fc1_dims=256, fc2_dims=256):
        super(PolicyGradientNetwork, self).__init__()
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions

        self.fc1 = Dense(self.fc1_dims, activation='relu')
        self.fc2 = Dense(self.fc2_dims, activation='relu')
        self.pi = Dense(n_actions, activation='softmax')

    def call(self, state):
        value = self.fc1(state)
        value = self.fc2(value)

        pi = self.pi(value)

        return pi


In [62]:
class Agent:
    def __init__(self, alpha=0.003, gamma=0.99, n_actions=4,
                 layer1_size=256, layer2_size=256):

        self.gamma = gamma
        self.lr = alpha
        self.n_actions = n_actions
        self.state_memory = []
        self.action_memory = []
        self.reward_memory = []
        self.policy = PolicyGradientNetwork(n_actions=n_actions)
        self.policy.compile(optimizer=Adam(learning_rate=self.lr))

    def choose_action(self, observation):
        state = tf.convert_to_tensor([observation], dtype=tf.float32)
        probs = self.policy(state)
        action_probs = tfp.distributions.Categorical(probs=probs)
        action = action_probs.sample()

        return action.numpy()[0]

    def store_transition(self, observation, action, reward):
        self.state_memory.append(observation)
        self.action_memory.append(action)
        self.reward_memory.append(reward)

    def learn(self):
        actions = tf.convert_to_tensor(self.action_memory, dtype=tf.float32)
        rewards = np.array(self.reward_memory)

        G = np.zeros_like(rewards)
        for t in range(len(rewards)):
            G_sum = 0
            discount = 1
            for k in range(t, len(rewards)):
                G_sum += rewards[k] * discount
                discount *= self.gamma
            G[t] = G_sum
        
        with tf.GradientTape() as tape:
            loss = 0
            for idx, (g, state) in enumerate(zip(G, self.state_memory)):
                state = tf.convert_to_tensor([state], dtype=tf.float32)
                probs = self.policy(state)
                action_probs = tfp.distributions.Categorical(probs=probs)
                log_prob = action_probs.log_prob(actions[idx])
                loss += -g * tf.squeeze(log_prob)

        gradient = tape.gradient(loss, self.policy.trainable_variables)
        self.policy.optimizer.apply_gradients(zip(gradient, self.policy.trainable_variables))

        self.state_memory = []
        self.action_memory = []
        self.reward_memory = []

In [79]:
if __name__ == '__main__':
    agent = Agent(alpha=0.0005,  gamma=0.99,n_actions=4)

    env = gym.make('LunarLander-v2')
    score_history = []

    num_episodes = 50

    for i in range(num_episodes):
        done = False
        score = 0
        observation = env.reset()
        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            agent.store_transition(observation, action, reward)
            observation = observation_
            score += reward
        score_history.append(score)

        agent.learn()
        avg_score = np.mean(score_history[-100:])
        print('episode: ', i,'score: %.1f' % score,
            'average score %.1f' % avg_score)

    filename = 'lunar-lander.png'
    

episode:  0 score: -153.7 average score -153.7
episode:  1 score: -206.3 average score -180.0
episode:  2 score: -247.2 average score -202.4
episode:  3 score: -224.4 average score -207.9
episode:  4 score: -223.4 average score -211.0
episode:  5 score: -106.9 average score -193.6
episode:  6 score: -516.6 average score -239.8
episode:  7 score: -322.3 average score -250.1
episode:  8 score: -254.3 average score -250.6
episode:  9 score: -126.4 average score -238.1
episode:  10 score: -92.7 average score -224.9
episode:  11 score: -150.3 average score -218.7
episode:  12 score: -258.1 average score -221.7
episode:  13 score: -60.0 average score -210.2
episode:  14 score: -78.2 average score -201.4
episode:  15 score: -119.1 average score -196.2
episode:  16 score: -247.0 average score -199.2
episode:  17 score: -16.7 average score -189.1
episode:  18 score: -48.8 average score -181.7
episode:  19 score: -35.6 average score -174.4
episode:  20 score: -70.1 average score -169.4
episode: 

In [ ]:
plotLearning(score_history, filename=filename, window=100)